In [4]:
import os
import sys
import inspect
import numpy as np

# add root path to access project modules
HERE_PATH = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(HERE_PATH)

# add the path of the wastebot package
WASTEBOT_PATH= os.path.join(HERE_PATH, '../library/wastebot')
sys.path.append(WASTEBOT_PATH)
from wastebot.heatbed.simulation import Heatbed

# <initialize a virtual heatbed>
# heatbed only version
# hb1_width = 0.20    # width of heatbed1 (unit: m)
# hb1_depth = 0.20    # depth of heatbed1 (unit: m)
# hb1_height = 0.0015 # height of heatbed1 (unit: m)

# heatbed and aluminum plate
hb1_width = 0.375    # width of heatbed1 (unit: m)
hb1_depth = 0.365    # depth of heatbed1 (unit: m)
hb1_height = 0.010 # height of heatbed1 (unit: m)

hb1_volume = hb1_width * hb1_depth * hb1_height # volume of heatbed1 (unit: m^3)
hb1_surf_area = hb1_width * hb1_depth * 2 + hb1_width * hb1_height * 2 + hb1_depth * hb1_height * 2 # surface area of heatbed1 (unit: m^2)
hb1_density = 2.7 * 10 ** 6 # density of heatbed1 (using the data of almiumum . unit: g/m^3)
hb1_sh = 0.88 # specific heat of heatbed1 (using the data of almiumum. unit: J/(g•K))
hb1_hc = hb1_sh  * hb1_density * hb1_volume # heat capacity of heatbed1 (unit: J/K)

hb1_initial_power = 13 # initail power of heatbed 1(unit: W)
hb1_max_power = 130.0 # max power of heatbed 1(unit: W)

hb1_initial_tmp = 20.0 # initial temperature of heatbed 1(unit: ℃)

# construct a virtual heatbed
hb1 = Heatbed(hb1_initial_tmp, hb1_hc, hb1_surf_area, hb1_initial_power, hb1_max_power)

# body of the simulation 
air_tmp = 20.0 # temperature of the air in the room (unit: ℃)
air_htc = 9.3 # heat transfer coefficient of the air in the room. (unit: W/(m^2 * K))
sampling_frequency = 10.0 # sampling frequency of simulation (unit: Hz)

number_of_steps = 20000 # number of sampling steps 

hb1_tmps = np.zeros(number_of_steps, dtype=float)

target_tmp = 37.0

controller_input = {"sv":target_tmp,
                                  "pv_range":np.array([10.0, 50.0]),
                                  "mv_max":hb1_max_power,
                                  "prop_band":10.0,
                                  "integral_time":1500.0,
                                  "derivative_time":0.0007,
                                  "initial_pv": hb1_initial_tmp}

from wastebot.heatbed import Controller
con1 = Controller(controller_input)

for i in xrange(number_of_steps):
    hb1_tmps[i] = hb1.tmp
    con1.pv = hb1.tmp
    hb1.power = con1.mv       
    hb1.update_tmp(1.0/sampling_frequency, air_tmp, air_htc)

# visualize
import matplotlib.pyplot as plt
t = np.arange(number_of_steps, dtype=float) / sampling_frequency
plt.plot(t, hb1_tmps)
plt.hlines(target_tmp, t[0], t[-1], linestyles="dashed")

c_ymin, c_ymax = plt.ylim()
if c_ymax < (target_tmp +1):
    plt.ylim([c_ymin, target_tmp + 1])

plt.xlabel("time[sec]")
plt.ylabel("temperature[degree]")
plt.title("'PID' control")

ImportError: No module named controller